# Generate Dataset

This notebook geenrates a dataset for a 4 DoF spatial robotic arm described below:
- 4 revolute joints with no limits
- 5 links  each 1 unit long

Our dataset consists of a set of features, $X$, and a set of targets $y$.
- $X$ consists of a 7 values. The first 3 represent a point $(x,y,z)$ and the last 4 represent an orientation in the form of a quaternion $(x,y,z,w)$
- $y$ consists of joint angles, with the amount of features being equal to the amount of joints.

### Adjust Path
In order to use the RobotArm class located in `inverse_kinematics_prediction/utils/robot_arm.py` we need to add this project to our system path.

In [1]:
import sys
import os

# Add the parent directory (project root) relative to the current working directory.
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Current working directory:", os.getcwd())
print("Project root added to sys.path:", project_root)

Current working directory: c:\Users\cjsta\git\inverse_kinematics_prediction\notebooks
Project root added to sys.path: c:\Users\cjsta\git\inverse_kinematics_prediction


### Create a RobotArm object
The RobotArm object defines the structure of a desired arm and can calculate the location of it's end effector.

In [2]:
import numpy as np

from utils.robot_arm import RobotArm

# Create a 4-DOF robot arm with 5 links, each of length 1.0 unit.
num_joints = 3
joint_limits = [(0, 2*np.pi)] * num_joints
rotation_axes = ['z', 'y', 'y']
link_lengths = [1.0] * (num_joints + 1)
link_axes = ['z', 'x', 'x', 'x']

# Instantiate the RobotArm object
robot_arm = RobotArm(num_joints=num_joints, 
                     joint_limits=joint_limits, 
                     link_lengths=link_lengths, 
                     rotation_axes=rotation_axes,
                     link_axes=link_axes)

# Test forward kinematics
test_angles = robot_arm.sample_random_joint_angles()
pos, quat = robot_arm.forward_kinematics(test_angles)
print("Joint angles (radians): ", test_angles)
print("End Effector Position (point x,y,z): ", pos)
print("End Effector Orientation (quaternion x,y,z,w): ", quat)

Joint angles (radians):  [1.15494794 0.26021044 2.6582621 ]
End Effector Position (point x,y,z):  [-0.39753814 -0.90021781  0.30016916]
End Effector Orientation (quaternion x,y,z,w):  (np.float64(-0.542515685970297), np.float64(0.8326359531464224), np.float64(0.06077541873462081), np.float64(0.09327619461448072))


### Generate Targets
Our target is the position and orientation of the end effector. We generate a uniformly random set of those. 

In [3]:
num_samples = 100000  # Parameter to control the number of samples

joint_samples = np.zeros((num_samples, robot_arm.num_joints))

# Generate random angles for each joint within its allowed range.
for i, (low, high) in enumerate(robot_arm.joint_limits):
    joint_samples[:, i] = np.random.uniform(low, high, size=num_samples)

print("Generated random joint samples shape:", joint_samples.shape)

Generated random joint samples shape: (100000, 3)


### Generate Features
Our features are the position and orientation of the end effector.



In [4]:
# Preallocate lists to store positions and quaternions
position_list = []
quaternion_list = []

# Loop through each sample and compute the forward kinematics
for joint_angles in joint_samples:
    pos, quat = robot_arm.forward_kinematics(joint_angles)
    position_list.append(pos)
    quaternion_list.append(quat)

# Convert the lists to numpy arrays
position_samples = np.array(position_list)       # Shape should be (100000, 3)
quaternion_samples = np.array(quaternion_list)     # Shape should be (100000, 4)

print("Position samples shape:", position_samples.shape)
print("Quaternion samples shape:", quaternion_samples.shape)

Position samples shape: (100000, 3)
Quaternion samples shape: (100000, 4)


### Save Dataset

In [5]:
import pandas as pd

# Dynamically create joint angle column names based on the number of joints.
joint_angle_columns = [f'joint_angle_{i+1}' for i in range(robot_arm.num_joints)]
position_columns = ['pos_x', 'pos_y', 'pos_z']
quaternion_columns = ['quat_x', 'quat_y', 'quat_z', 'quat_w']

# Concatenate the lists to form the final list of columns.
columns = joint_angle_columns + position_columns + quaternion_columns

# Concatenate the arrays along the columns
data = np.concatenate((joint_samples, position_samples, quaternion_samples), axis=1)

# Create a DataFrame with the data and the dynamically generated column names.
df = pd.DataFrame(data, columns=columns)

# Create the target directory if it doesn't exist.
target_dir = os.path.join(project_root, "data", "raw")
os.makedirs(target_dir, exist_ok=True)

# Define the file path; you might update the file name if the DOF changes.
file_path = os.path.join(target_dir, "dataset.csv")
df.to_csv(file_path, index=False)

print("CSV file saved as", file_path)

CSV file saved as c:\Users\cjsta\git\inverse_kinematics_prediction\data\raw\dataset.csv
